In [27]:
import cv2

background = cv2.imread("layout3.png")
background = (background * 1).astype("uint8")

objectA = cv2.imread("objectA.png", -1)
width, height, channels = background.shape[:3]
object_height = objectA.shape[0]

start_pos = ([height*0.49,height*0.495,height*0.5,height*0.505,height*0.51], width*0.15)
end_pos = ([height*0.27,height*0.39,height*0.5,height*0.61,height*0.73], width*0.75)

total_frames = 300
plus_frames = 60

dy = (end_pos[1] - start_pos[1]) / plus_frames

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output = cv2.VideoWriter("output.mp4", fourcc, 60.0, (background.shape[1], background.shape[0]),True)

def empty_frame(total_frames):
    for i in range(total_frames):
        background_copy = background.copy()
        output.write(background_copy)
        output.release()

def write_notes(total_frames,timing,lane):
    for i in range(total_frames):
        if timing <= i < timing + plus_frames:
            dx = (end_pos[0][lane-1] - start_pos[0][lane-1]) / plus_frames
            size_ratio = 0.2 + (i-timing) / plus_frames * 0.8
            objectA_resized = cv2.resize(objectA, (0, 0), fx=size_ratio, fy=size_ratio)
            x = int(start_pos[0][lane-1] + (i-timing) * dx - objectA_resized.shape[0]/2)
            y = int(start_pos[1] + (i-timing) * dy - objectA_resized.shape[1]/2)
            background_copy = background.copy()
            roi = background_copy[y:y+objectA_resized.shape[0], x:x+objectA_resized.shape[1]]
            alpha = objectA_resized[:,:,3] / 255.0
            for c in range(3):
                roi[:, :, c] = alpha * objectA_resized[:, :, c] + (1 - alpha) * roi[:, :, c]
            background_copy[y:y+objectA_resized.shape[0], x:x+objectA_resized.shape[1]] = roi
        else:
            background_copy = background.copy()
        output.write(background_copy)
    output.release()

# empty_frame(total_frames)
write_notes(total_frames,60,1)


output.release()
cv2.destroyAllWindows()
